<a href="https://colab.research.google.com/github/SD2711/Python_university/blob/main/Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Проект 1: Консольный калькулятор с меню и историей операций**

📝 Техническое задание:
Разработайте интерактивное консольное приложение «Калькулятор», которое:

Предоставляет пользователю текстовое меню с выбором математических операций: сложение, вычитание, умножение, деление, возведение в степень, остаток от деления, целочисленное деление.

Принимает числовые значения от пользователя и выполняет выбранную операцию.
Сохраняет историю всех вычислений (в виде строк, например: “5 + 3 = 8”) и позволяет в любой момент:

Просмотреть всю историю.

Очистить историю.

Обеспечивает корректную обработку ошибок: деление на ноль, ввод нечисловых данных, неверный выбор пункта меню.

Работает в цикле до явного выбора пользователем команды «Выход».Каждая новая функциональная часть (меню, операции, история, обработка ошибок) фиксируется отдельным коммитом в репозитории.
---



In [2]:
# main.py
import json
import csv
import sqlite3
from datetime import datetime

class Calculator:
    def __init__(self):
        self.history = []
        self.init_database()

    def init_database(self):
        """Инициализация SQLite базы данных для хранения истории"""
        self.conn = sqlite3.connect('calculator_history.db', check_same_thread=False)
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                operation TEXT NOT NULL,
                result REAL NOT NULL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        self.conn.commit()

    def add_to_history(self, operation, result):
        """Добавление операции в историю (в память и в БД)"""
        history_entry = f"{operation} = {result}"
        self.history.append(history_entry)

        # Сохранение в SQLite
        cursor = self.conn.cursor()
        cursor.execute(
            "INSERT INTO history (operation, result) VALUES (?, ?)",
            (operation, result)
        )
        self.conn.commit()

    def get_history_from_db(self):
        """Получение истории из базы данных"""
        cursor = self.conn.cursor()
        cursor.execute("SELECT operation, result FROM history ORDER BY timestamp DESC")
        return [f"{row[0]} = {row[1]}" for row in cursor.fetchall()]

    def clear_history(self):
        """Очистка истории (из памяти и БД)"""
        self.history.clear()
        cursor = self.conn.cursor()
        cursor.execute("DELETE FROM history")
        self.conn.commit()
        print("✅ История очищена!")

    def display_history(self):
        """Отображение истории операций"""
        if not self.history:
            print("📝 История пуста")
            return

        print("\n📊 История операций:")
        print("-" * 40)
        for i, entry in enumerate(self.get_history_from_db(), 1):
            print(f"{i}. {entry}")
        print("-" * 40)

    def export_to_json(self, filename="calculator_history.json"):
        """Экспорт истории в JSON файл"""
        try:
            history_data = []
            cursor = self.conn.cursor()
            cursor.execute("SELECT operation, result, timestamp FROM history")

            for row in cursor.fetchall():
                history_data.append({
                    'operation': row[0],
                    'result': row[1],
                    'timestamp': row[2]
                })

            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(history_data, f, ensure_ascii=False, indent=2)

            print(f"✅ История экспортирована в {filename}")
        except Exception as e:
            print(f"❌ Ошибка при экспорте в JSON: {e}")

    def get_number_input(self, prompt):
        """Безопасный ввод числа с обработкой ошибок"""
        while True:
            try:
                return float(input(prompt))
            except ValueError:
                print("❌ Ошибка: Введите корректное число!")

    def perform_operation(self, operation_type):
        """Выполнение математической операции"""
        try:
            if operation_type in ['+', '-', '*', '/', '%', '//', '**']:
                a = self.get_number_input("Введите первое число: ")
                b = self.get_number_input("Введите второе число: ")

                if operation_type == '+':
                    result = a + b
                    operation_str = f"{a} + {b}"
                elif operation_type == '-':
                    result = a - b
                    operation_str = f"{a} - {b}"
                elif operation_type == '*':
                    result = a * b
                    operation_str = f"{a} * {b}"
                elif operation_type == '/':
                    if b == 0:
                        raise ZeroDivisionError("Деление на ноль!")
                    result = a / b
                    operation_str = f"{a} / {b}"
                elif operation_type == '%':
                    if b == 0:
                        raise ZeroDivisionError("Деление на ноль!")
                    result = a % b
                    operation_str = f"{a} % {b}"
                elif operation_type == '//':
                    if b == 0:
                        raise ZeroDivisionError("Деление на ноль!")
                    result = a // b
                    operation_str = f"{a} // {b}"
                elif operation_type == '**':
                    result = a ** b
                    operation_str = f"{a} ** {b}"

                print(f"🎯 Результат: {operation_str} = {result}")
                self.add_to_history(operation_str, result)

        except ZeroDivisionError as e:
            print(f"❌ Ошибка: {e}")
        except Exception as e:
            print(f"❌ Неожиданная ошибка: {e}")

    def display_menu(self):
        """Отображение главного меню"""
        print("\n" + "=" * 50)
        print("🧮 КОНСОЛЬНЫЙ КАЛЬКУЛЯТОР")
        print("=" * 50)
        print("1. ➕ Сложение")
        print("2. ➖ Вычитание")
        print("3. ✖️ Умножение")
        print("4. ➗ Деление")
        print("5. 🔢 Остаток от деления")
        print("6. 🔍 Целочисленное деление")
        print("7. 💪 Возведение в степень")
        print("8. 📊 Просмотр истории")
        print("9. 🗑️ Очистить историю")
        print("10. 💾 Экспорт в JSON")
        print("0. 🚪 Выход")
        print("-" * 50)

    def run(self):
        """Основной цикл программы"""
        print("Добро пожаловать в калькулятор!")

        while True:
            self.display_menu()

            try:
                choice = input("Выберите операцию (0-11): ").strip()

                if choice == '0':
                    print("👋 До свидания!")
                    break
                elif choice == '1':
                    self.perform_operation('+')
                elif choice == '2':
                    self.perform_operation('-')
                elif choice == '3':
                    self.perform_operation('*')
                elif choice == '4':
                    self.perform_operation('/')
                elif choice == '5':
                    self.perform_operation('%')
                elif choice == '6':
                    self.perform_operation('//')
                elif choice == '7':
                    self.perform_operation('**')
                elif choice == '8':
                    self.display_history()
                elif choice == '9':
                    self.clear_history()
                elif choice == '10':
                    self.export_to_json()
                else:
                    print("❌ Ошибка: Неверный выбор! Попробуйте снова.")

            except KeyboardInterrupt:
                print("\n👋 Программа прервана пользователем. До свидания!")
                break
            except Exception as e:
                print(f"❌ Неожиданная ошибка: {e}")

        # Закрытие соединения с БД при выходе
        self.conn.close()

# Запуск приложения
if __name__ == "__main__":
    calculator = Calculator()
    calculator.run()

Добро пожаловать в калькулятор!

🧮 КОНСОЛЬНЫЙ КАЛЬКУЛЯТОР
1. ➕ Сложение
2. ➖ Вычитание
3. ✖️ Умножение
4. ➗ Деление
5. 🔢 Остаток от деления
6. 🔍 Целочисленное деление
7. 💪 Возведение в степень
8. 📊 Просмотр истории
9. 🗑️ Очистить историю
10. 💾 Экспорт в JSON
0. 🚪 Выход
--------------------------------------------------
Выберите операцию (0-11): 4
Введите первое число: 2.3
Введите второе число: 2.4
🎯 Результат: 2.3 / 2.4 = 0.9583333333333333

🧮 КОНСОЛЬНЫЙ КАЛЬКУЛЯТОР
1. ➕ Сложение
2. ➖ Вычитание
3. ✖️ Умножение
4. ➗ Деление
5. 🔢 Остаток от деления
6. 🔍 Целочисленное деление
7. 💪 Возведение в степень
8. 📊 Просмотр истории
9. 🗑️ Очистить историю
10. 💾 Экспорт в JSON
0. 🚪 Выход
--------------------------------------------------
Выберите операцию (0-11): 0
👋 До свидания!
